In [1]:
import yfinance as yf

In [27]:
import requests
import pandas as pd
from io import BytesIO

def get_gics_classification(trading_day: str) -> pd.DataFrame:
    """
    KRX API를 통해 GICS 산업분류(산업별현황) 테이블을 가져옵니다.
    
    Args:
        trading_day (str): 조회 기준일 ('YYYYMMDD' 형식)
    Returns:
        pd.DataFrame: GICS Sector, Industry Group, Industry, Sub-Industry 등 분류 정보
    """
    # 1) OTP 생성 요청 URL
    gen_otp_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
    # 2) OTP 요청 파라미터 (네트워크 탭에서 확인한 'url' 값으로 변경)
    gen_otp_data = {
        # trading_day는 대부분의 기본 통계 메뉴에서 필수 파라미터입니다
        "trdDd": trading_day,
        # name, csvxls_isNo 고정
        "name": "fileDown",
        "csvxls_isNo": "false",
        # 여기를 실제 GICS 산업분류용 서비스 코드로 바꿔주세요
        "url": "dbms/MDC/MDI/industry/MDCMDI03401"
    }
    # 3) Referer 헤더 (해당 메뉴 URL)
    headers = {
        "Referer": "https://index.krx.co.kr/Index/Index?menuId=MDC0201010401"
    }
    # OTP(일회용 토큰) 받기
    otp = requests.post(gen_otp_url, data=gen_otp_data, headers=headers).text

    # 4) 받은 OTP로 CSV 다운로드
    download_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
    resp = requests.post(download_url, data={"code": otp}, headers=headers)
    # EUC-KR 인코딩된 CSV를 DataFrame으로 변환
    df = pd.read_csv(BytesIO(resp.content), encoding="euc-kr")
    return df

if __name__ == "__main__":
    # 예: 20250807 기준 데이터
    gics_df = get_gics_classification("20250807")
    print(gics_df.head())

                                        <HTML><HEAD>
0                       <TITLE>Access Denied</TITLE>
1                                      </HEAD><BODY>
2                             <H1>Access Denied</H1>
3  You don't have permission to access "http&#58;...
4  Reference&#32;&#35;18&#46;26ad2c17&#46;1754550...


In [34]:
# 1) OTP 생성 요청 URL
trading_day = "20250807"

gen_otp_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
# 2) OTP 요청 파라미터 (네트워크 탭에서 확인한 'url' 값으로 변경)
gen_otp_data = {
    # trading_day는 대부분의 기본 통계 메뉴에서 필수 파라미터입니다
    "trdDd": trading_day,
    # name, csvxls_isNo 고정
    "name": "fileDown",
    "csvxls_isNo": "false",
    # 여기를 실제 GICS 산업분류용 서비스 코드로 바꿔주세요
    "url": "dbms/MDC/MDI/industry/MDCMDI03401"
}
headers = {
    "Referer": "https://index.krx.co.kr/Index/Index?menuId=MDC0201010401"
}
otp = requests.post(gen_otp_url, data=gen_otp_data, headers=headers).text
otp

download_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
resp = requests.post(download_url, data={"code": otp}, headers=headers)
df = pd.read_csv(BytesIO(resp.content), encoding="euc-kr")
# EUC-KR 인코딩된 CSV를 DataFrame으로 변환

In [35]:
df.head()

,<HTML><HEAD>
0,<TITLE>Access Denied</TITLE>
1,</HEAD><BODY>
2,<H1>Access Denied</H1>
3,"You don't have permission to access ""http&#58;..."
4,Reference&#32;&#35;18&#46;8e023517&#46;1754550...


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import time
warnings.filterwarnings("ignore")

pages = 5;
code = '005930'  # Samsung Electronics

def get_total_pages(code, pages=pages):
    results = []
    page = 1
    last_date = None

    for page in range(1, pages+1):
        url = f"https://finance.naver.com/item/sise_day.nhn?code={code}&page={page}"
        res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, verify=False)
        soup = BeautifulSoup(res.text, 'html.parser')
        table = soup.find('table', class_='type2')
        df = pd.read_html(str(table))[0]
        df = df.dropna()
        if df.empty:
            break
        results.append(df)
        # 종료 조건: 마지막 페이지에 도달하면 같은 날짜가 반복됨(더 이상 새로운 데이터 없음)
        cur_last_date = df.iloc[-1, 0]
        if last_date == cur_last_date:
            break
        last_date = cur_last_date
        page += 1
        time.sleep(0.5)  # 과도한 요청 방지

    data = pd.concat(results)
    data = data.dropna()
    
    colnames = data.columns.tolist()
    print(colnames)
    # 컬럼명 한글 → 영문 변환
    data.columns = [
        'Date', 'Close', 'Change', 'Volume',
        'Foreign_Holdings', 'Foreign_Holdings_Ratio',
        'Foreign_Net', 'Institution_Net', 'Individual_Net', 'Total_Net'
    ]
    data['Date'] = pd.to_datetime(data['Date'])
    data = data.sort_values(by='Date', ascending=False).reset_index(drop=True)
    return data
    
results = get_total_pages(code,5)
results.head(5)
print(f"수집된 전체 일수: {len(results)}")


['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']


ValueError: Length mismatch: Expected axis has 7 elements, new values have 10 elements

In [26]:
pages = 2
code = '005930'  # Samsung Electronics

all_data = []
for page in range(1, pages + 1):
    url = f"https://finance.naver.com/item/frgn.naver?code={code}&page={page}"
    res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, verify=False)
    soup = BeautifulSoup(res.text, "html.parser")
    table = soup.find('table', class_='type2')
    df = pd.read_html(str(table))[0]
    all_data.append(df)
    time.sleep(0.5)

    data = pd.concat(all_data)
    data = data.dropna()
    
colnames = data.columns.tolist()

# 실제 컬럼 개수/이름 확인 (필요하면 print로 점검)
print(data.columns.tolist())

['매도상위', '거래량', '매수상위', '거래량.1']


In [18]:
 print(data.columns.tolist())

['매도상위', '거래량', '매수상위', '거래량.1']
